In [38]:
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [39]:
df=sns.load_dataset('tips')

In [40]:
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [41]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


#### ques. predict the time? weather it is lunch or dinner?
##### time is the atrget variable

In [42]:
df.time.unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [43]:
# eda >> subjective  

#encoding , missing value treatment,scaling--- now we will automate this

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [45]:
df.isnull().sum()
#still we will see if there is a null value how can we treat it

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

##### since time is nomial so we will use label encoder

In [46]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [47]:
df['time'].unique()

array([0, 1])

In [48]:
x=df.drop('time',axis=1)
y=df['time']

In [49]:
x

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


In [50]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int64

In [51]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

In [52]:
#now we will do missing value treatment , feature scaling and data encoding

In [53]:
from sklearn.impute import SimpleImputer #for missing value treatment
from sklearn.preprocessing import OneHotEncoder #for encoding
from sklearn.preprocessing import StandardScaler #for scaling


from sklearn.pipeline import Pipeline # sequence of data tranformer
from sklearn.compose import ColumnTransformer #groups all the pipeline steps for each of the columns

In [54]:
df.sample(1)

,total_bill,tip,sex,smoker,day,time,size
100,11.35,2.5,Female,Yes,Fri,0,2


In [55]:
cat_col=['sex','smoker','day']
num_col=['total_bill','tip','size']

### feature engineering automation using pipeline and column transformer

In [56]:
num_pipe=Pipeline(steps=[('imputation',SimpleImputer(strategy='median')),
                ('scaling',StandardScaler())])

In [57]:
cat_pipe=Pipeline(steps=[('imputation',SimpleImputer(strategy='most_frequent')),
                ('encoding',OneHotEncoder())])

In [58]:
prepro=ColumnTransformer([('num_pipeline',num_pipe,num_col),
                   ('cat_pipeline',cat_pipe,cat_col)])

In [59]:
prepro

,transformers,"[('num_pipeline', ...), ('cat_pipeline', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'median'
,fill_value,None


In [60]:
x_train=prepro.fit_transform(x_train)
x_test=prepro.transform(x_test)

In [61]:
# now data is ready now we can build multiple models 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [62]:
models={'svm':SVC(),
        'dt':DecisionTreeClassifier(),
        'logis':LogisticRegression()}

In [63]:
from sklearn.metrics import accuracy_score


In [64]:
for i in range(len(models)):
    print(list(models.values())[i])

SVC()
DecisionTreeClassifier()
LogisticRegression()


In [65]:
def model_train_evl(x_train,x_test,y_train,y_test,models):
    evaluation={}
    for i in range(len(models)):
        model=list(models.values())[i]
        model.fit(x_train,y_train)
        y_pred=model.predict(x_test)
        model_score=accuracy_score(y_test,y_pred)
        evaluation[list(models.keys())[i]]=model_score

    return evaluation

In [66]:
model_train_evl(x_train,x_test,y_train,y_test,models)

{'svm': 0.9183673469387755,
 'dt': 0.9183673469387755,
 'logis': 0.9183673469387755}